## Baseline experiments eith ESN

In [1]:
from tabulate import tabulate
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, LMS, RLS, Ridge, ESN)
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import copy
import math

In [2]:
# Define basic parameters
units = 1000
lr = 0.3
sr = 0.8
ridge = 1e-5
delay = 5
order = 2
strides = 1

In [ ]:
numEvals = 20
trainSize = 3000
teacherForcing = 1000

def testMGSDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

print(testMGSDeepCascadeNVAR())

In [10]:
# Define basic parameters
units = 1000
lr = 0.3
sr = 0.8
ridge = 1e-5
delay = 5
order = 2
strides = 1

# Datasets
datasets = {
    # 'lorenz': {
    #     'data': lorenz(n_timesteps=3000),
    #     'warmup': 100,
    #     'test': 100
    # },
    'mackey_glass': {
        'data': mackey_glass(n_timesteps=5000),
        'warmup': 1000,
        'test': 2000
    },
    # 'narma': {
    #     'data': narma(n_timesteps=3000),
    #     'warmup': 100,
    #     'test': 100
    # }
}

results = {}

# Evaluation
for dataset_name, config in datasets.items():
    if dataset_name == 'lorenz':
        output_dim = 3
    else:
        output_dim = 1

    # Models
    models = {
        'ESN': (Reservoir(units=units, lr=lr, sr=sr), Ridge(output_dim=output_dim, ridge=ridge)),
        'ESN-IP': (IPReservoir(units=units, lr=lr, sr=sr), Ridge(output_dim=output_dim, ridge=ridge)),
        'ESN-LMS': (Reservoir(units=units, lr=lr, sr=sr), LMS(output_dim=output_dim)),
        'ESN-RLS': (Reservoir(units=units, lr=lr, sr=sr), RLS(output_dim=output_dim)),
        'NVAR-ESN': (NVAR(delay=delay, order=order, strides=strides), Ridge(output_dim=output_dim, ridge=ridge))
    }

    X = config['data'][:-1]
    y_true = config['data'][1:]

    X_train = X[:len(X)-config['test']]
    X_test = X[len(X)-config['test']:]
    y_train = y_true[:len(y_true)-config['test']]
    y_test = y_true[len(y_true)-config['test']:]

    print(X_train)

    for model_name, (reservoir, readout) in models.items():
        esn = reservoir >> readout

        if isinstance(readout, (LMS, RLS)):
            esn.train(X_train, y_train)
        else:
            esn.fit(X_train, y_train, warmup=config['warmup'])

        y_pred = esn.run(X_test)

        mse_val = mse(y_test, y_pred)
        rmse_val = rmse(y_test, y_pred)
        rsquare_val = rsquare(y_test, y_pred)
        nrmse_val = nrmse(y_test, y_pred)
        
        results[f"{dataset_name} - {model_name}"] = {
            "RMSE": rmse_val,
            "R^2": rsquare_val,
            "NRMSE": nrmse_val,
            "MSE": mse_val
        }

# Prepare data for the table
table_data = []
headers = ["Dataset - Model", "RMSE", "R^2", "NRMSE", "MSE"]

for result_key, result_values in results.items():
    row = [result_key]
    for metric in headers[1:]:
        row.append(result_values[metric])
    table_data.append(row)

# Print the table
print(tabulate(table_data, headers=headers))


[[1.2       ]
 [1.13549691]
 [1.05735972]
 ...
 [0.94510214]
 [0.93300303]
 [0.92314809]]


Running Model-19: 2999it [00:00, 9344.36it/s]<?, ?it/s]
Running Model-19: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]


Fitting node Ridge-21...


Running Model-19: 2000it [00:00, 10272.00it/s]         
Running Model-20: 100%|██████████| 1/1 [00:00<00:00, 11.67it/s]


Fitting node IPReservoir-7...


...

Fitting node Ridge-22...


Running Model-20: 2000it [00:00, 9330.01it/s]          
Running Model-21: 2000it [00:00, 10182.25it/s]         
Running Model-22: 2000it [00:00, 10381.56it/s]         
Running Model-23: 2999it [00:00, 29146.89it/s]?, ?it/s]
Running Model-23: 100%|██████████| 1/1 [00:00<00:00,  9.44it/s]


Fitting node Ridge-23...


Running Model-23: 2000it [00:00, 28373.63it/s]         

Dataset - Model                 RMSE        R^2        NRMSE          MSE
-----------------------  -----------  ---------  -----------  -----------
mackey_glass - ESN       0.00170224   0.999944   0.00182878   2.89761e-06
mackey_glass - ESN-IP    0.00138663   0.999963   0.00148972   1.92276e-06
mackey_glass - ESN-LMS   0.619756     0.0187428  0.66583      0.384098
mackey_glass - ESN-RLS   0.00134043   0.999965   0.00144009   1.79676e-06
mackey_glass - NVAR-ESN  0.000437417  0.999996   0.000469936  1.91334e-07


In [6]:

numEvals = 20
trainSize = 3000
teacherForcing = 1000

def testMGSRelaxation(esn):
    data = np.array(mackey_glass(n_timesteps=(trainSize)))
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)
    # Relaxation stage
    esnPreds = copy.deepcopy(esn).run(X, forced_feedbacks=Y)
    esnPreds[0] = Y[0]
    esn.fit(X, esnPreds, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=(teacherForcing+85), seed=i))
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)

        pred = esnToTest.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGS(esn):
    data = np.array(mackey_glass(n_timesteps=(trainSize)))
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=(teacherForcing+85), seed=i))
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)

        pred = esnToTest.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGSProposed(esn):
    data = np.array(mackey_glass(n_timesteps=(trainSize)))
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=(teacherForcing+85), seed=i))
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

models = {
    'ESN': Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5),
}

for model_name, model in models.items():
    error1 = testMGS(copy.deepcopy(model))
    error2 = testMGSProposed(copy.deepcopy(model))
    error3 = testMGSRelaxation(copy.deepcopy(model))
    print("==================Lorenz NRMSE==================")
    print('Normal training error:', error1)
    print('Proposed method error:', error2)
    print('Relaxation method error:', error3)

Running Model-8: 2999it [00:00, 15631.60it/s]?, ?it/s]
Running Model-8: 100%|██████████| 1/1 [00:00<00:00,  4.78it/s]


Fitting node Ridge-8...


Running Model-8: 1000it [00:00, 13494.14it/s]         
Running Model-8: 84it [00:00, 15107.48it/s]           
Running Model-8: 1000it [00:00, 13136.39it/s]         
Running Model-8: 84it [00:00, 12900.83it/s]           
Running Model-8: 1000it [00:00, 13786.80it/s]         
Running Model-8: 84it [00:00, 14809.03it/s]           
Running Model-8: 1000it [00:00, 13447.98it/s]         
Running Model-8: 84it [00:00, 15227.62it/s]           
Running Model-8: 1000it [00:00, 13576.17it/s]         
Running Model-8: 84it [00:00, 13997.68it/s]           
Running Model-8: 1000it [00:00, 13964.49it/s]         
Running Model-8: 84it [00:00, 13999.90it/s]           
Running Model-8: 1000it [00:00, 13331.71it/s]         
Running Model-8: 84it [00:00, 16802.82it/s]           
Running Model-8: 1000it [00:00, 13189.76it/s]         
Running Model-8: 84it [00:00, 14599.77it/s]           
Running Model-8: 1000it [00:00, 13852.87it/s]         
Running Model-8: 84it [00:00, 13957.75it/s]           
Running Mo

Fitting node Ridge-8...


Running Model-8: 2999it [00:00, 16774.51it/s]         
Running Model-9: 2999it [00:00, 15635.70it/s]?, ?it/s]
Running Model-9: 100%|██████████| 1/1 [00:00<00:00,  4.66it/s]


...

Fitting node Ridge-8...


Running Model-8: 2999it [00:00, 13090.46it/s]         
Running Model-8: 2999it [00:00, 15859.01it/s]?, ?it/s]
Running Model-8: 100%|██████████| 1/1 [00:00<00:00,  4.79it/s]


Fitting node Ridge-8...


Running Model-8: 1000it [00:00, 13458.16it/s]         
Running Model-8: 84it [00:00, 15083.55it/s]           
Running Model-8: 1000it [00:00, 13483.04it/s]         
Running Model-8: 84it [00:00, 15361.74it/s]           
Running Model-8: 1000it [00:00, 13752.72it/s]         
Running Model-8: 84it [00:00, 15259.28it/s]           
Running Model-8: 1000it [00:00, 13483.95it/s]         
Running Model-8: 84it [00:00, 14012.15it/s]           
Running Model-8: 1000it [00:00, 13626.54it/s]         
Running Model-8: 84it [00:00, 15254.00it/s]           
Running Model-8: 1000it [00:00, 13868.13it/s]         
Running Model-8: 84it [00:00, 13409.51it/s]           
Running Model-8: 1000it [00:00, 13920.41it/s]         
Running Model-8: 84it [00:00, 14009.92it/s]           
Running Model-8: 1000it [00:00, 14219.23it/s]         
Running Model-8: 84it [00:00, 12828.95it/s]           
Running Model-8: 1000it [00:00, 13753.21it/s]         
Running Model-8: 84it [00:00, 16778.01it/s]           
Running Mo

==================Lorenz NRMSE==================
Normal training error: 0.0007938887075682848
Proposed method error: 0.0003412351233293849
Relaxation method error: 0.0011363628614734435


In [58]:
numEvals = 20
trainSize = 3000
teacherForcing = 2000

def testMGSRelaxation():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)
    # Relaxation stage
    esnPreds = copy.deepcopy(esn).run(X, forced_feedbacks=Y)
    esnPreds[0] = Y[0]
    esn.fit(X, esnPreds, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)

        pred = esnToTest.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGS():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)

        pred = esnToTest.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGSDeep():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)

        pred = esnToTest.run(x_test)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGSDeepCascade():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

def testMGSCascade():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))


def testMGSDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))

error1 = testMGS()
error2 = testMGSCascade()
error3 = testMGSRelaxation()
error4 = testMGSDeep()
error5 = testMGSDeepCascade()
error6 = testMGSDeepCascadeNVAR()
print("==================Lorenz NRMSE==================")
print('Relaxation method error:', error3)
print('Normal training error:', error1)
print('Cascading ESN error:', error2)
print('Deep ESN error:', error4)
print('Deep + cascade error:', error5)
print('Deep + cascade + NVAR error:', error6)

Running Model-185: 2999it [00:00, 17305.77it/s]?, ?it/s]
Running Model-185: 100%|██████████| 1/1 [00:00<00:00,  5.23it/s]


Fitting node Ridge-124...


Running Model-185: 2000it [00:00, 14916.10it/s]         
Running Model-185: 84it [00:00, 14303.99it/s]           
Running Model-185: 2000it [00:00, 14734.82it/s]         
Running Model-185: 84it [00:00, 16538.59it/s]           
Running Model-185: 2000it [00:00, 14433.63it/s]         
Running Model-185: 84it [00:00, 15505.06it/s]           
Running Model-185: 2000it [00:00, 14659.32it/s]         
Running Model-185: 84it [00:00, 17492.75it/s]           
Running Model-185: 2000it [00:00, 14436.14it/s]         
Running Model-185: 84it [00:00, 15939.27it/s]           
Running Model-185: 2000it [00:00, 14988.30it/s]         
Running Model-185: 84it [00:00, 15798.46it/s]           
Running Model-185: 2000it [00:00, 14893.46it/s]         
Running Model-185: 84it [00:00, 16714.34it/s]           
Running Model-185: 2000it [00:00, 14891.29it/s]         
Running Model-185: 84it [00:00, 15099.06it/s]           
Running Model-185: 2000it [00:00, 14545.68it/s]         
Running Model-185: 84it [00:00,

Fitting node Ridge-125...


Running Model-186: 2999it [00:00, 17106.99it/s]         
Running Model-187: 2999it [00:00, 16732.94it/s]?, ?it/s]
Running Model-187: 100%|██████████| 1/1 [00:00<00:00,  4.95it/s]


...

Fitting node Ridge-131...


Running Model-196: 2999it [00:00, 9028.15it/s]          
Running Model-198: 2999it [00:00, 7561.34it/s]<?, ?it/s]
Running Model-198: 100%|██████████| 1/1 [00:00<00:00,  2.38it/s]


Fitting node Ridge-132...


Running Model-196: 2000it [00:00, 7409.55it/s]          
Running Model-198: 2000it [00:00, 7603.98it/s]          
Running Model-196: 84it [00:00, 9041.77it/s]            
Running Model-198: 84it [00:00, 6975.14it/s]            
Running Model-196: 2000it [00:00, 7485.41it/s]          
Running Model-198: 2000it [00:00, 7467.13it/s]          
Running Model-196: 84it [00:00, 8988.94it/s]            
Running Model-198: 84it [00:00, 6916.40it/s]            
Running Model-196: 2000it [00:00, 7245.84it/s]          
Running Model-198: 2000it [00:00, 7349.38it/s]          
Running Model-196: 84it [00:00, 8894.76it/s]            
Running Model-198: 84it [00:00, 8459.91it/s]            
Running Model-196: 2000it [00:00, 7344.99it/s]          
Running Model-198: 2000it [00:00, 7326.67it/s]          
Running Model-196: 84it [00:00, 7671.84it/s]            
Running Model-198: 84it [00:00, 6411.56it/s]            
Running Model-196: 2000it [00:00, 7444.58it/s]          
Running Model-198: 2000it [00:0

==================Lorenz NRMSE==================
Relaxation method error: 0.0011436340439655343
Normal training error: 0.0009319866483285244
Cascading ESN error: 0.00034120102538657605
Deep ESN error: 0.00027397618465792295
Deep + cascade error: 0.00014394263326449698
Deep + cascade + NVAR error: 3.77149096845285e-05


In [48]:
def testMGSDeepCascadeNVAR():
    esn = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    data = np.array(mackey_glass(n_timesteps=(10 * trainSize), h=0.1))
    data = data[::10]
    data = np.tanh(data-1)
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    esn2 = (NVAR(delay=8, order=3, strides=4) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(X)
    esn2.fit(esnPreds, Y, warmup=100)

    error = 0
    for i in range(numEvals):
        data2 = np.array(mackey_glass(n_timesteps=10 * (teacherForcing+85), seed=i, h=0.1))
        data2 = data2[::10]
        data2 = np.tanh(data2-1)

        esnToTest = copy.deepcopy(esn)
        esn2Copy = copy.deepcopy(esn2)

        teacherForcingX = data2[:teacherForcing]
        teacherForcingY = data2[1:teacherForcing+1]
        x_test = data2[teacherForcing:-1]
        y_test = data2[teacherForcing+84:]
        outputs = esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        esn2Copy.run(outputs, forced_feedbacks=teacherForcingY)

        esnPreds = esnToTest.run(x_test)
        pred = esn2Copy.run(esnPreds)[-1]
        error += (y_test[0][0] - pred)**2
    variance = np.var(data)
    return math.sqrt(error/(numEvals*variance))
print(testMGSDeepCascadeNVAR())

Running Model-148: 2999it [00:00, 9207.52it/s]<?, ?it/s]
Running Model-148: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]


Fitting node Ridge-105...


Running Model-148: 2999it [00:00, 9642.99it/s]          
Running Model-150: 2999it [00:00, 9175.91it/s]<?, ?it/s]
Running Model-150: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


Fitting node Ridge-106...


Running Model-148: 1000it [00:00, 8488.88it/s]          
Running Model-150: 1000it [00:00, 8709.41it/s]          
Running Model-148: 84it [00:00, 8855.64it/s]            
Running Model-150: 84it [00:00, 10492.94it/s]           
Running Model-148: 1000it [00:00, 8706.32it/s]          
Running Model-150: 1000it [00:00, 8335.26it/s]          
Running Model-148: 84it [00:00, 10497.32it/s]           
Running Model-150: 84it [00:00, 10266.38it/s]           
Running Model-148: 1000it [00:00, 8789.52it/s]          
Running Model-150: 1000it [00:00, 8352.94it/s]          
Running Model-148: 84it [00:00, 9325.86it/s]            
Running Model-150: 84it [00:00, 10468.00it/s]           
Running Model-148: 1000it [00:00, 8548.20it/s]          
Running Model-150: 1000it [00:00, 8407.20it/s]          
Running Model-148: 84it [00:00, 9331.29it/s]            
Running Model-150: 84it [00:00, 8843.41it/s]            
Running Model-148: 1000it [00:00, 8412.67it/s]          
Running Model-150: 1000it [00:0

3.0296918485504927e-05


In [61]:
def testLorenzRefined(esn):
    data = np.array([[0.01 * x[0]] for x in lorenz(n_timesteps=6000, h=0.02)])
    noise = np.random.uniform(-1e-8, 1e-8, data.shape)
    data += noise
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    # Relaxation stage
    esnPreds = copy.deepcopy(esn).run(X, forced_feedbacks=Y)
    esnPreds[0] = Y[0]
    esn.fit(X, esnPreds, warmup=1000)

    error = 0
    numEvals = 20
    for i in range(numEvals):
        data2 = np.array([[0.01 * x[0]] for x in lorenz(n_timesteps=1685, x0=[1+0.01*i, 1, 1], h=0.02)])
        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:1000]
        teacherForcingY = data2[1:1001]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        pred = esnToTest.run(data2[1000:])[-1] * 100
        actual = data2[-1] * 100
        error += (actual - pred)**2

    variance = np.var(100*data)
    return math.sqrt(error/(numEvals*variance))

def testLorenz(esn):
    data = np.array([[0.01 * x[0]] for x in lorenz(n_timesteps=6000, h=0.02)])
    noise = np.random.uniform(-1e-8, 1e-8, data.shape)
    data += noise
    X = data[:-1]
    Y = data[1:]

    esn.fit(X, Y, warmup=1000)

    error = 0
    numEvals = 20
    for i in range(numEvals):
        data2 = np.array([[0.01 * x[0]] for x in lorenz(n_timesteps=1685, x0=[1+0.01*i, 1, 1], h=0.02)])
        esnToTest = copy.deepcopy(esn)

        teacherForcingX = data2[:1000]
        teacherForcingY = data2[1:1001]
        esnToTest.run(teacherForcingX, forced_feedbacks=teacherForcingY)
        pred = esnToTest.run(data2[1000:])[-1][-1] * 100
        actual = data2[-1][-1] * 100
        error += (actual - pred)**2
    variance = np.var(100*data)
    print(error, variance)
    return math.sqrt(error/(numEvals*variance))

models = {
    'ESN': Reservoir(units=500, sr=0.97, rc_connectivity=0.02) >> Ridge(output_dim=1, ridge=1e-5),
    # 'ESN-IP': IPReservoir(units=units, lr=lr, sr=sr) >> Ridge(output_dim=output_dim, ridge=ridge),
    # 'NVAR-ESN': NVAR(delay=delay, order=order, strides=strides) >> Ridge(output_dim=output_dim, ridge=ridge)
}

for model_name, model in models.items():
    error2 = testLorenz(model)
    print("==================Lorenz NRMSE==================")
    print(error2)

Running Model-39: 5999it [00:00, 19925.92it/s]?, ?it/s]
Running Model-39: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]


Fitting node Ridge-39...


Running Model-39: 1000it [00:00, 17757.28it/s]         
Running Model-39: 685it [00:00, 21422.81it/s]          
Running Model-39: 1000it [00:00, 17844.61it/s]         
Running Model-39: 685it [00:00, 20789.72it/s]          
Running Model-39: 1000it [00:00, 17229.95it/s]         
Running Model-39: 685it [00:00, 21397.44it/s]          
Running Model-39: 1000it [00:00, 17097.14it/s]         
Running Model-39: 685it [00:00, 21229.84it/s]          
Running Model-39: 1000it [00:00, 16947.23it/s]         
Running Model-39: 685it [00:00, 21850.48it/s]          
Running Model-39: 1000it [00:00, 17769.31it/s]         
Running Model-39: 685it [00:00, 21149.20it/s]          
Running Model-39: 1000it [00:00, 17714.30it/s]         
Running Model-39: 685it [00:00, 21044.79it/s]          
Running Model-39: 1000it [00:00, 17525.48it/s]         
Running Model-39: 685it [00:00, 20883.11it/s]          
Running Model-39: 1000it [00:00, 17444.36it/s]         
Running Model-39: 685it [00:00, 21396.64it/s]   

13.130913542713529 62.611245042595456
==================Lorenz NRMSE==================
0.10240149641288099


In [36]:
def getData():
    data = np.array(mackey_glass(n_timesteps=8000))
    train = data[:5600]
    test = data[5600:]
    trainX = train[:-1]
    trainY = train[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, testX, testY

def testDefault():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testNVAR():
    esn = NVAR(delay=5, order=2, strides=1) >> Ridge(output_dim=output_dim, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeep():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)
    preds = esn.run(testX)
    return nrmse(testY, preds)

def testDeepCascade():
    esn = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = (Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)

def testCascade():
    esn = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    trainX, trainY, testX, testY = getData()

    esn.fit(trainX, trainY, warmup=1000)

    esn2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10) >> Ridge(output_dim=1, ridge=1e-5)
    esnPreds = copy.deepcopy(esn).run(trainX)
    esn2.fit(esnPreds, trainY, warmup=1000)

    preds1 = esn.run(testX)
    preds2 = esn2.run(preds1)
    return nrmse(testY, preds2)


def evaluateModel(model, trainX, trainY, valX, valY):
    if model.is_trained_online:
        model.train(trainX, trainY)
    else:
        model.fit(trainX, trainY, warmup=min(int(len(trainX)/10), 100))
    nodePreds = model.run(valX)
    if isinstance(nodePreds, np.ndarray):
        return nrmse(nodePreds, valY)
    else:
        bestError = np.inf
        for node in nodePreds:
            preds = nodePreds[node]
            bestError = min(bestError, nrmse(preds, valY))
        return bestError


def testDeepCascadeNVAR():
    nvar1 = NVAR(delay=8, order=3, strides=4)
    reservoir1 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    readout1 = Ridge(output_dim=1, ridge=1e-5)
    nvar2 = NVAR(delay=8, order=3, strides=4)
    reservoir2 = Reservoir(units=1000, sr=0.8, rc_connectivity=0.01, noise_in=1e-10)
    readout2 = Ridge(output_dim=1, ridge=1e-5)
    model = nvar1 >> reservoir1 >> readout1 >> nvar2 >> reservoir2 >> readout2
    trainX, trainY, testX, testY = getData()

    return evaluateModel(model, trainX, trainY, testX, testY)

# error1 = testDefault()
# error2 = testCascade()
# error4 = testDeep()
# error5 = testDeepCascade()
error6 = testDeepCascadeNVAR()
error7 = testNVAR()
print("==================MGS==================")
# print('Normal training error:', error1)
# print('Cascading ESN error:', error2)
# print('Deep ESN error:', error4)
# print('Deep + cascade error:', error5)
print('Deep + cascade + NVAR error:', error6)
print('NVAR training error:', error7)

Running Model-120: 5599it [00:00, 7759.60it/s]<?, ?it/s]
Running Model-120: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


Fitting node Ridge-61...


Running Model-120: 5599it [00:00, 7081.11it/s]<?, ?it/s]
Running Model-120: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


Fitting node Ridge-62...


Running Model-120: 2399it [00:00, 4825.53it/s]          
Running Model-121: 5599it [00:00, 28281.16it/s]?, ?it/s]
Running Model-121: 100%|██████████| 1/1 [00:00<00:00,  4.99it/s]


Fitting node Ridge-63...


Running Model-121: 2399it [00:00, 29495.70it/s]         

==================MGS==================
Deep + cascade + NVAR error: 0.0006214651086084345
NVAR training error: 0.0040074829371010935
